Todo:
- čísla týdnů (52+53 atd)
- správně jména! 
    - Žbirka Miro aj.

In [3]:
import os

In [4]:
import pandas as pd

In [5]:
folder = "data_raw"

In [6]:
df = pd.read_csv(os.path.join(folder,"ifpi_raw.csv"))

In [7]:
df

,Unnamed: 0,hitparada,obdobi,pozice,track/album,interpret,vydavatel
0,0,CZ - ALBUMS - TOP 100,1. týden 2006,1,Album,PETR KOLÁŘ,UNIVERSAL MUSIC
1,1,CZ - ALBUMS - TOP 100,1. týden 2006,2,Music Bar,CHINASKI,UNIVERSAL MUSIC
2,2,CZ - ALBUMS - TOP 100,1. týden 2006,3,Čím to je?!,NO NAME,UNIVERSAL MUSIC
3,3,CZ - ALBUMS - TOP 100,1. týden 2006,4,44 slavných písniček,NEDVĚDI,UNIVERSAL MUSIC
4,4,CZ - ALBUMS - TOP 100,1. týden 2006,5,Best Of 1988-2005,PAVOL HABERA & TEAM,UNIVERSAL MUSIC
...,...,...,...,...,...,...,...
496801,45,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,46,Naše cesty,MAREK ZTRACENÝ,SUPRAPHON
496802,46,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,47,Naše hriechy (feat. Emma Drobná),ADAM ĎURICA,UNIVERSAL MUSIC
496803,47,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,48,Náš příběh (feat. Marek Lambora),TEREZA MAŠKOVÁ,WARNER MUSIC
496804,48,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,49,Patříme k sobě,OLGA LOUNOVÁ,OLGA LOUNOVÁ RECORDS


In [8]:
hitparady = df['hitparada'].drop_duplicates().to_list()
hitparady

['CZ - ALBUMS - TOP 100',
 'CZ - RADIO - TOP 100',
 'CZ - RADIO - TOP 20 MODERN ROCK',
 'CZ - ALBUMS CLASSICAL - TOP 10',
 'CZ - SINGLES DIGITAL - TOP 100',
 'CZ - SINGLES DIGITAL - TOP 50  CZ',
 'SK - ALBUMS - TOP 100',
 'SK - RADIO - TOP 100',
 'SK - RADIO - TOP 50 SK',
 'SK - SINGLES DIGITAL - TOP 100',
 'SK - SINGLES DIGITAL - TOP 50 SK',
 'CZ - RADIO - TOP 50 CZ']

## Opravy a čištění

### Aby byla data strojově čitelná

In [9]:
df['tyden'] = df['obdobi'].apply(lambda x: x.split(". t")[0] if ". t" in x else None)

In [10]:
df['tyden'] = df['tyden'].apply(lambda x: f"0{x}" if x and len(x) == 1 else x)

In [11]:
df['mesic'] = df['obdobi'].apply(lambda x: x.split("/")[0].split(" ")[1] if "Měsíc" in x else None)

In [12]:
df['mesic'] = df['mesic'].apply(lambda x: f"0{x}" if x and len(x) == 1 else x)

In [13]:
df['rok'] = df['obdobi'].apply(lambda x: x[-4:])

In [14]:
df.groupby('tyden')['track/album'].nunique().nsmallest(15)

tyden
5152       188
200903     190
200809     198
50.+51     206
53         328
52.+53     487
27.+28     519
30.+31     519
52.+01     524
52        1036
51        1785
51.+52    2072
50        4002
01        4108
31        4108
Name: track/album, dtype: int64

### Oprava chybných dat

In [15]:
df["rok"] = df["rok"].apply(lambda x: x.replace("223","2023").replace(" 201","2010").replace("2104","2014").strip())

### Týdny dvouciferně

In [16]:
df['tyden'].drop_duplicates().to_list()

['01',
 '02',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '28',
 '27',
 '29',
 '30',
 None,
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '5152',
 '03',
 '04',
 '51',
 '200809',
 '52',
 '200903',
 '51.+52',
 '50.+51',
 '53',
 '30.+31',
 '52.+01',
 '52.+53',
 '27.+28']

In [17]:
def cistetydny(x):
    
    try:
        
        x = x.replace(".","").replace("+","")
        if len(x) == 2:
            t = x
        else:
            if x[0:2] != "52":
                t = x[-2:]
            else:
                t = "52"
    
    except:
        
        t = x
    
    return t

In [18]:
cistetydny('200809')

'09'

In [19]:
df['tyden'] = df['tyden'].apply(lambda x: cistetydny(x))

In [20]:
df['tyden'].drop_duplicates().to_list()

['01',
 '02',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '28',
 '27',
 '29',
 '30',
 None,
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '52',
 '03',
 '04',
 '51',
 '53']

### Doplnění týdne

In [21]:
def weekuj(rok, tyden):
    try:
        return f"""{rok}-{tyden:02}"""
    except:
        return None

In [22]:
df['week'] = df.apply(lambda x: weekuj(x['rok'], x['tyden']), axis = 1)

In [23]:
df['week']

0         2006-01
1         2006-01
2         2006-01
3         2006-01
4         2006-01
           ...   
496801       None
496802       None
496803       None
496804       None
496805       None
Name: week, Length: 496806, dtype: object

### Pryč s dvojitými mezerami

In [24]:
df = df.apply(lambda x:x.replace("  "," "))

### Pryč s velkými písmeny (nebo jejich většinou)

In [25]:
slovodelice = [" ",".","/"]
for s in slovodelice:
    df['interpret'] = df['interpret'].apply(lambda x: s.join(word.title() for word in str(x).split(s)))
    df['vydavatel'] = df['vydavatel'].apply(lambda x: s.join(word.title() for word in str(x).split(s)))

In [26]:
df.sample(5)

,Unnamed: 0,hitparada,obdobi,pozice,track/album,interpret,vydavatel,tyden,mesic,rok,week
110879,29,CZ - RADIO - TOP 100,2. týden 2013,30,Don´t Stop The Party,Pitbull & Tjr,Sony Music,02,None,2013,2013-02
319439,70,SK - ALBUMS - TOP 100,36. týden 2023,71,Nevermind (20th Anniversary Edition),Nirvana,Universal Music,36,None,2023,2023-36
110157,7,CZ - RADIO - TOP 100,46. týden 2012,8,Some Nights,Fun.,Warner Music,46,None,2012,2012-46
20819,19,CZ - ALBUMS - TOP 100,9. týden 2014,20,Swing Both Ways,Robbie Williams,Universal Music,09,None,2014,2014-09
134209,59,CZ - RADIO - TOP 100,31. týden 2017,60,Fetish,Selena Gomez & Gucci Mane,Universal Music,31,None,2017,2017-31


### Sjednocení jmen

In [27]:
df[df['interpret'].str.lower().str.contains("vířat",na=False)]['interpret'].drop_duplicates().to_list()

['Sto Zvířat']

In [28]:
spravna_jmena = {
    "Ondřej G. Brzobohatý": "Ondřej Brzobohatý",
    "M. Cyrus": "Miley Cyrus",
    "Ulrychovi Hana & Petr": "Hana A Petr Ulrychovi",
    "Ulrychovci":  "Hana A Petr Ulrychovi",
    "Miroslav Žbirka": "Miro Žbirka",
    "P.Hapka": "Petr Hapka",
    "Natalie Grossová": "Natálie Grossová",
    "Tichá Kateřina Marie": "Kateřina Marie Tichá",
    "Al Dhaba Gába": "Gabriela Al Dhábba",
    "Křížek Láďa": "Ladislav Křížek",
    "AC/DC": "ACDC",
    " A Jiní": "",
    " A ": " a "
}

In [29]:
interpreti = df['interpret'].drop_duplicates().to_list()
[x for x in interpreti if len(x.split()) > 1 and f"{x.split()[1]} {x.split()[0]}" in interpreti]

['Nedvěd František',
 'Simon Paul',
 'Jones Norah',
 'Baumaxa Xavier',
 'Manu Chao',
 'Gahan  Dave',
 'Lipa Peter',
 'Duran Duran',
 'Sodoma Viktor',
 'Pospíšil Luboš & 5P',
 'Ward Shayne',
 'Toxxx Hugo',
 'Tobias Szidi',
 'Cohen Avishai',
 'Petřina Ota',
 'Janů Petra',
 'Pop Iggy',
 'Hrůza Michal & Ready Kirken',
 'Streisand Barbra',
 'Satriani Joe',
 'Bárta Dan & Robert Balzar Trio',
 'Knechtová Katarína & Peha',
 'Absolonová Monika',
 'Hybš Václav',
 'Spálený Jan',
 'Šporcl Pavel & Romano Stilo',
 'Ruml Ondřej',
 'Merta Vladimír & Etc',
 'Havelka Ondřej',
 'Dragoun Roman',
 'Smith  Patti',
 'Young Neil & Crazy Horse',
 'Brown Chris',
 'Mercury Freddie & Caballé Monserrat',
 'Horáček Michal & Lucia Šoralová',
 'Havelka Ondřej A Jeho Melody Makers',
 'Merta Vladimír & Dobrá Úroda',
 'Lang Lang',
 'Dědeček Jiří',
 'Bruni Carla',
 'Mišík Vladimír & Kovářík Mirek & Čdg',
 'Bonamassa Joe & Beth Hart',
 'Zagorová Hana / Petr Rezek',
 'Müller Richard & Fragile',
 'Petty Tom',
 'Tony Bennett

In [30]:
obracena_jmena = ["Bílá Lucie","Žbirka Miro","Vondráčková Lucie", "Jiřina Anna Jandová", "Smolík Jakub", "Sofian Medjmedj", "Josef Bohouš", "Müller Richard","Rolins Dara", "Zmožek Marcel", "Hrůza Michal", "Deyl David", "Bende Petr", "Zonyga Jiří", "Hapka Petr", "Nedvěd František", "Mrózek Daniel", "Issa Sámer","Koller David","Ondra Jakub", "Smejkal Jaro", "Vondráčková Helena", "Střihavka Kamil", "Charvátová Jitka", "Goldová Gabriela","Přenosilová Yvonne", "Machálková Leona","Prokop Michal","Suchý Jiří","Langerová Aneta","Daniel Šafařík", "Jandová Marta", "Horváth Vlastimil", "Horecká Hana", "Farna Ewa", "John Wolfhooker", "Adam Janota",  "Dyk Vojtěch", "Jíšová Pavlína", "Sanchez Yvonne", "Ruppert Matěj", "Burian Michael", "Brzobohatá Katka", "Václav Rouček", "Šinkorová Daniela", "Šeps Michal", "Absolonová Monika", "Zíma Josef", "Rouček Václav","Schmitzer Jiří","Ztracený Marek", "Zemanová Bára", "Bastien Alan", "Maxa Martin", "Matuš Bohuš", "Hron Vláďa", "Hudček Michal", "Háma Aleš", "Hübsch Kamila", "Hůlka Daniel", "Milfajtová Sára", "Krajčo Richard", "Kraus David", "Kreyson Memorial", "Krézlová Karolína", "Josef Vágner, Charlotte Gottová", "Hladík Radim", "Peter Fider", "Drda Zbyněk", "Paul Project", "Povýšilová Monika", "Poláček Petr", "Poliak Michal", "Hanák Tomáš", "Hayek Martin", "Nekonečný Dan", "Neckář Václav", "Novotný Miloš","Rosová Sandra","Vojtek Roman","Bagová Ivanna","Kerndlová Tereza","Kotvald Petr",'Simon Paul',"Albarn Damon",
 'Jones Norah',
 'Baumaxa Xavier',
 'Chao Manu',
 'Gahan Dave',
 'Lipa Peter',
 'Pospíšil Luboš',
 'Sodoma Viktor',
 'Ward Shayne',
 'Toxxx Hugo',
 'Tobias Szidi',
 'Cohen Avishai',
 'Petřina Ota',
 'Janů Petra',
 'Pop Iggy',
 'Streisand Barbra',
 'Satriani Joe',
 'Bárta Dan',
 'Knechtová Katarína',
 'Hybš Václav',
 'Spálený Jan',
 'Šporcl Pavel',
 'Ruml Ondřej',
 'Merta Vladimír',
 'Havelka Ondřej',
 'Dragoun Roman',
 'Smith Patti',
 'Young Neil',
 'Brown Chris',
 'Mercury Freddie',
 'Caballé Monserrat',
 'Horáček Michal',
 'Havelka Ondřej',
 'Merta Vladimír',
 'Dědeček Jiří',
 'Bruni Carla',
 'Mišík Vladimír',
 'Bonamassa Joe',
 'Zagorová Hana',
 'Petty Tom',
 'Bennett Tony',
 'Dusilová Lenka',
 'Ondřej Havelka',
 'Ruml Ondřej',
 'Szidi Tobias',
 'Ondřej Havelka',
 'Roman Dragoun And His Angels',
 'Avishai Cohen Trio',
 'Sinatra Frank',
 'Zdeněk Bína',
 'Melody Gardot',
 'Petřina Ota',
 'Cooper Alice',
 'Satriani Joe',
 'Veverka Vilém',
 'Bernstein Leonard',
 'Dragoun Roman',
 'Clyro Biffy',
 'Gardot Melody',
 'Spálený Jan',
 'Hybš Václav',
 'Michal Hromek',
 'Bernstein Leonard',
 'Veverka Vilém',
 'Svěcený Jaroslav',
 'Semerádová Jana',
 'Ramazzotti Eros & Anastacia',
 'Foret Michael',
 'Ward Shayne',
 'Keys Alicia & White Jack',
 'Burke Alexandra Feat. Flo Rida',
 'Bieber Justin',
 'Tempah Tinie',
 'Williams Robbie',
 'Iglesias Enrique',
 'Solveig Martin',
 'Lopez Jennifer',
 'Sinclar Bob',
 'Posner Mike',
 'Brown Chris',
 'Solveig Martin',
 'Lopez Jennifer',
 'Brown Chris',
 'Gomez Selena',
 'Murs Olly',
 'Harris Calvin',
 'Burke Alexandra',
 'Konvičková Markéta',
 'Harris Calvin',
 'Grey Skylar',
 'Romero Nicky',
 'Solveig Martin',
 'Derulo Jason',
 'Tempah Tinie',
 'Lavigne Avril',
 'Harris Calvin',
 'Bublé Michael',
 'Aristone Peter',
 'Garand Paulie',
 'Azalea Iggy',
 'Paul Sean',
 'Ponte Gabry',
 'Cooper Alice',
 'Bína Zdeněk',
 'Bezdeda Tomáš',
 'Opatovský Robo',
 'Kerndlová Tereza',
 'Kmeťo Igor',
 'Lipa Peter',
 'Tomáš Bezdeda',
 'Mirgová Dominika',
                 'Gahan Dave',
                 'Cohen Avishai',
                 'Smith Patti',
                 'Dragoun Roman',
                 'Gott Karel',
                 'Savka Tomáš',
                 'Schmitzer Jiří',
                 'Cristovao Ben',
                 "Norisová Zuzana"]

In [31]:
obracena_jmena

['Bílá Lucie',
 'Žbirka Miro',
 'Vondráčková Lucie',
 'Jiřina Anna Jandová',
 'Smolík Jakub',
 'Sofian Medjmedj',
 'Josef Bohouš',
 'Müller Richard',
 'Rolins Dara',
 'Zmožek Marcel',
 'Hrůza Michal',
 'Deyl David',
 'Bende Petr',
 'Zonyga Jiří',
 'Hapka Petr',
 'Nedvěd František',
 'Mrózek Daniel',
 'Issa Sámer',
 'Koller David',
 'Ondra Jakub',
 'Smejkal Jaro',
 'Vondráčková Helena',
 'Střihavka Kamil',
 'Charvátová Jitka',
 'Goldová Gabriela',
 'Přenosilová Yvonne',
 'Machálková Leona',
 'Prokop Michal',
 'Suchý Jiří',
 'Langerová Aneta',
 'Daniel Šafařík',
 'Jandová Marta',
 'Horváth Vlastimil',
 'Horecká Hana',
 'Farna Ewa',
 'John Wolfhooker',
 'Adam Janota',
 'Dyk Vojtěch',
 'Jíšová Pavlína',
 'Sanchez Yvonne',
 'Ruppert Matěj',
 'Burian Michael',
 'Brzobohatá Katka',
 'Václav Rouček',
 'Šinkorová Daniela',
 'Šeps Michal',
 'Absolonová Monika',
 'Zíma Josef',
 'Rouček Václav',
 'Schmitzer Jiří',
 'Ztracený Marek',
 'Zemanová Bára',
 'Bastien Alan',
 'Maxa Martin',
 'Matuš Bohuš'

In [32]:
for o in obracena_jmena:
    spravna_jmena[o] = f"{o.split()[1]} {o.split()[0]}"

In [33]:
for key, val in spravna_jmena.items():
    df['interpret'] = df['interpret'].apply(lambda x: x.replace(key,val))

In [34]:
interpreti = df['interpret'].drop_duplicates().to_list()
[x for x in interpreti if len(x.split()) > 1 and f"{x.split()[1]} {x.split()[0]}" in interpreti]

['Gahan  Dave',
 'Duran Duran',
 'Avishai Cohen',
 'Roman Dragoun',
 'Smith  Patti',
 'Lang Lang',
 'Dragoun Roman',
 'Cohen Avishai',
 'Laurie Anderson & Kronos Quartet',
 'Jan Vodňanský a Haňa Navarová',
 'Guetta David Vs. The Egg',
 'Guetta David Feat. Rowland Kelly ',
 'Guetta David Feat. Akon',
 'Guetta David Feat. Fergie',
 'Guetta David Feat. Rihanna',
 'Guetta David Feat. Flo-Rida & Nicki Minaj',
 'Guetta David Feat.Taio Cruz & Ludacris',
 'Guetta David Feat. Sia',
 'Guetta David Feat. Usher',
 'David Michal a Lucie Vondráčková',
 'Guetta David & Minaj Nicki',
 'Guetta David Feat. Chris Brown And Lil Wayne',
 'Timberlake Justin & Jay-Z',
 'Guetta David Feat. Taped Rai',
 'Guetta David & Ne-Yo & Akon',
 'Guetta David Feat. Skylar Grey',
 'Junge Junge & Kyle Pearce',
 'Junge Junge',
 'Junge Junge & Joe Cleere',
 'Loko Loko & Rytmus',
 'Fifty Fifty',
 'Loko Loko',
 'Keith Urban Feat. P!Nk',
 'Loko Loko & Momo',
 'Loko Loko & Hard Rico']

### Hlavní intepreti

In [35]:
df[df['interpret'].str.contains(" a ")]['interpret'].drop_duplicates()

86                  Jaroslav Uhlíř a Zdeněk Svěrák
390                          Hana a Petr Ulrychovi
874                                  Mňága a Žďorp
9590                          Ryvolovi Wabi a Miky
12990                               Žalman a Spol.
14050                 Robert Křesťan a Druhá Tráva
17784          Havelka Ondřej a Jeho Melody Makers
21084                      Michnová Zuzana a Hosté
23460                   Wabi Daněk a Ďáblovo Stádo
24147                  Petr Hapka a Michal Horáček
28567                  Věra Špinarová a Petra Janů
30456                           Laura a Její Tygři
34097                     Michal Prokop a Framus 5
58233                       Y.Simonová a M.Chladil
60492                Jan Vodňanský a Haňa Navarová
76104                             System Of a Down
88042                                Marek a Kamil
104510            David Michal a Lucie Vondráčková
108147                   Markéta Konvičková a Elis
110710    Eddie Stoilow Feat. E

In [36]:
def kdojeprvni(x):
    delice = ["&","/","(feat.","feat.","Feat."," a "]
    for d in delice:
        if d in x:
            x = x.split(d)[0]
    return x.strip()

In [37]:
df['interpret_hlavni'] = df['interpret'].apply(lambda x: kdojeprvni(x))

In [38]:
df['interpret_hlavni'].drop_duplicates().to_list()

['Petr Kolář',
 'Chinaski',
 'No Name',
 'Nedvědi',
 'Pavol Habera',
 'Aneta Langerová',
 'Těžkej Pokondr',
 'Robbie Williams',
 'Damichi',
 'Divokej Bill',
 'Petr Novák',
 'Různí',
 'Vlasta Horváth',
 'Rammstein',
 'Madonna',
 'Petr Bende',
 'Helena Zeťová',
 'Jakub Smolík',
 'Čechomor',
 'Richard Müller',
 'Tatu',
 'Karel Gott',
 'Depeche Mode',
 'Petr Muk',
 'Lenka Filipová',
 'Michal David',
 'Black Eyed Peas',
 'Anastacia',
 'Juanes',
 'Amiri Ali',
 'Sámer Issa',
 'Tři Sestry',
 'Věra Špinarová',
 'Enya',
 'Karel Kryl',
 'Maxim Turbulenc',
 'Miro Žbirka',
 'Zuzana Navarová',
 'Iveta Bartošová',
 'Marie Rottrová',
 'Pražský Výběr',
 'Hana Hegerová',
 'Dan Bárta',
 'Iva Frühlingová',
 'Jiří Schelinger',
 'Leoš Mareš',
 'Janek Ledecký',
 'Eminem',
 'Jaroslav Uhlíř',
 'Petr Hapka',
 'Lucie Vondráčková',
 'Alanis Morissette',
 '4Tet',
 'Jana Kirschner',
 'James Blunt',
 'Crazy Frog',
 'Lenka Dusilová',
 'Victoria',
 'Hana Zagorová',
 'Marika Gombitová',
 'Radůza',
 '2M',
 'Peha',
 'Mic

In [39]:
len(df[(df['hitparada'] == 'CZ - RADIO - TOP 100')]['interpret_hlavni'].drop_duplicates().to_list())

1434

### Album/track

In [40]:
df['album'] = df.apply(lambda row: row['track/album'] if 'ALBUM' in row['hitparada'] else None, axis = 1)

In [41]:
df['track'] = df.apply(lambda row: row['track/album'] if 'ALBUM' not in row['hitparada'] else None, axis = 1)

In [42]:
df = df.drop(columns=['track/album','obdobi'])

In [43]:
poradi = ['hitparada','rok','mesic','tyden','week','pozice','album','track','interpret','interpret_hlavni','vydavatel',]

In [44]:
df = df.reindex(columns=poradi)

### Seřazení

In [45]:
df = df.sort_values(by=['hitparada','rok','mesic','tyden','pozice'])

In [46]:
df.sample(10)

,hitparada,rok,mesic,tyden,week,pozice,album,track,interpret,interpret_hlavni,vydavatel
322953,SK - RADIO - TOP 100,2017,None,22,2017-22,85,None,Beautiful Ones,Hurts,Hurts,Sony Music
318561,SK - ALBUMS - TOP 100,2023,None,29,2023-29,93,TRUSTFALL,None,P!Nk,P!Nk,Sony Music
464314,CZ - RADIO - TOP 50 CZ,2013,None,18,2013-18,41,None,Přímý zásah (2012),David Deyl,David Deyl,Supraphon
329150,SK - RADIO - TOP 100,2018,None,32,2018-32,82,None,Perfect Strangers,Jonas Blue,Jonas Blue,Universal Music
312068,SK - ALBUMS - TOP 100,2022,06,None,None,100,Medusa II,None,Sergei Barracuda,Sergei Barracuda,Skerocuda
383139,SK - SINGLES DIGITAL - TOP 100,2016,None,47,2016-47,60,None,Final Song,Mo,Mo,Sony Music
479999,CZ - RADIO - TOP 50 CZ,2018,None,22,2018-22,44,None,Šňůry,Kryštof,Kryštof,I.Richmond.P./Universal Music
117463,CZ - RADIO - TOP 100,2014,None,17,2014-17,14,None,Say Something,A Great Big World & Christina Aguilera,A Great Big World,Sony Music
11787,CZ - ALBUMS - TOP 100,2010,None,31,2010-31,38,Jsem,None,Aneta Langerová,Aneta Langerová,Sony Music
49952,CZ - ALBUMS - TOP 100,2019,None,43,2019-43,3,Ta o mně,None,Lucie Bílá,Lucie Bílá,Supraphon


## Oprava chybných oprav (nutno zde)

In [51]:
df.loc[df['interpret'] == 'Nico & Vinz',"interpret_hlavni"]="Nico & Vinz"
df.loc[df['interpret'] == 'Zagorová',"interpret_hlavni"]="Hana Zagorová"

### Export do jednotlivých dataframů

In [48]:
import os

if not os.path.exists("data"):
    os.mkdir("data")
else:
    pass

In [49]:
df

,hitparada,rok,mesic,tyden,week,pozice,album,track,interpret,interpret_hlavni,vydavatel
1850,CZ - ALBUMS - TOP 100,2006,05,None,None,1,Divokej Bill,None,Divokej Bill,Divokej Bill,Warner Music
1851,CZ - ALBUMS - TOP 100,2006,05,None,None,2,Stadium Arcadium,None,Red Hot Chili Peppers,Red Hot Chili Peppers,Warner Music
1852,CZ - ALBUMS - TOP 100,2006,05,None,None,3,Asi se něco děje,None,Ready Kirken,Ready Kirken,Universal Music
1853,CZ - ALBUMS - TOP 100,2006,05,None,None,4,Pearl Jam,None,Pearl Jam,Pearl Jam,Sony Music
1854,CZ - ALBUMS - TOP 100,2006,05,None,None,5,Schrei,None,Tokio Hotel,Tokio Hotel,Universal Music
...,...,...,...,...,...,...,...,...,...,...,...
445321,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,39,2023-39,46,None,NEVÍŠ NIC pt. 2,Yzomandias,Yzomandias,Milion+ Ent./Universal Music
445322,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,39,2023-39,47,None,Nočná,Separ,Separ,Ruka Hore
445323,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,39,2023-39,48,None,VIDÍM JAK SA NA MŇA POZERÁŠ (feat. Ego),Pil C,Pil C,Universal Music
445324,SK - SINGLES DIGITAL - TOP 50 SK,2023,None,39,2023-39,49,None,Chcem mať s tebou bejby,Pil C,Pil C,Universal Music


In [50]:
for h in hitparady:
    df_p = df[df["hitparada"] == h].reset_index(drop=True)
    if "ALBUMS" in h:
        df_p = df_p.drop(columns=['track'])
    else:
        df_p = df_p.drop(columns=['album'])
    df_p2 = df_p[~df_p['mesic'].isnull()]
    h = h.replace(" ","_").replace("-_","")
    if len(df_p2) > 0:
        df_p2 = df_p2.drop(columns=['tyden'])
        df_p2.to_csv(os.path.join("data",f"{h}_mesicni.csv"))
    df_p2 = df_p[~df_p['tyden'].isnull()]
    if len(df_p2) > 0:
        df_p2 = df_p2.drop(columns=['mesic'])
        df_p2.to_csv(os.path.join("data",f"{h}_tydenni.csv"))